Auteur: Michaël Leroy

    Création  geodataframe des Noeuds du graph:
        attributs:
          * osmid : id openstreetmap de ref de la commune  <- index
          * codegeo: code insee pour pouvoir requeter sur les autres datasets
          * y: longitude point central de la commune
          * x: lattitude point centra de la commune	

          * counts: décompte du nombre de tags label 1 contenus dans le polygone de la commune
          * geometry: area shape de la commune

        ( A reprendre ou a requeter dans les datasets consolidés ?)
            pop:  nombre d'habitants
            bornes: 
            puissances:
            ....

  Les edges sont crées entre le centre commune et les 5 plus proches centres voisins (BallTree sklearn, distance Haversine), mais sans attributs à ce stade


  [TODO] vérifier si il ne faut pas mettre (v,u,1) quand (u,v,0)
          existe pour tenir compte de l'eventuelle non réciprocité des trajets routier
          en fonction des voies empruntées. En théorie pas nécessaire dataframe avec multi index,
          mais....


In [ ]:
class CFG:
    area = [
        'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]
    area_name = 'france'      #'_'.join(area)

    

    bornes = False

    viz = True

    roads = (
        '|motorway'
        '|motorway_link'
        '|trunk'
        '|trunk_link'
        '|primary'
        '|primary_link'
        '|secondary'
        '|secondary_link'
        '|tertiary'
         '|tertiary_link'
    )

        # '|primary'
        # '|motorway_link'
        
        # '|primary_link'

        # '|secondary'
        # '|secondary_link'
        
        
        #  '|tertiary'
        #  '|tertiary_link'

    data_path = './osm_datas/'

import joblib
joblib.dump(CFG.area, 'CFG.pkl')    

import os
os.environ['USE_PYGEOS'] = '0'

%load_ext autoreload
%autoreload


%aimport OSM_helper_functions
%aimport OCM_helper_functions

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import osmnx as ox
import networkx as nx
from tqdm.notebook import tqdm
from OSM_helper_functions import *
from OCM_helper_functions import *


ox.settings.log_console=True
ox.settings.use_cache=True
''' To retrieve http data from OSMnx and cache them locally
    before building graph. Raise and exception if requests
    are all done. intercept then set to False and retry.'''
# ox.settings.cache_only_mode=True 

Administratives = {
    'Auvergne-Rhône-Alpes': ['Ain', 'Allier', 'Ardèche', 'Cantal', 'Drôme', 'Isère', 'Loire', 'Haute-Loire', 'Puy-de-Dôme', 'Rhône', 'Savoie', 'Haute-Savoie'],
    'Bourgogne-Franche-Comté': ['Côte-d\'Or', 'Doubs', 'Jura', 'Nièvre', 'Haute-Saône', 'Saône-et-Loire', 'Yonne', 'Territoire de Belfort'],
    'Brittany': ['Côtes-d\'Armor', 'Finistère', 'Ille-et-Vilaine', 'Morbihan'],
    'Centre-Val de Loire': ['Cher', 'Eure-et-Loir', 'Indre', 'Indre-et-Loire', 'Loir-et-Cher', 'Loiret'],
    # 'Corse': ['Corse-du-Sud', 'Haute-Corse'],
    'Grand Est': ['Ardennes', 'Aube', 'Marne', 'Haute-Marne', 'Meurthe-et-Moselle', 'Meuse', 'Moselle', 'Bas-Rhin', 'Haut-Rhin', 'Vosges'],
    'Hauts-de-France': ['Aisne', 'Nord', 'Oise', 'Pas-de-Calais', 'Somme'],
    'Île-de-France': ['Paris', 'Seine-et-Marne', 'Yvelines', 'Essonne', 'Hauts-de-Seine', 'Seine-Saint-Denis', 'Val-de-Marne', 'Val-d\'Oise'],
    'Normandie': ['Calvados', 'Eure', 'Manche', 'Orne', 'Seine-Maritime'],
    'Nouvelle-Aquitaine': ['Charente', 'Charente-Maritime', 'Corrèze', 'Creuse', 'Dordogne', 'Gironde', 'Landes', 'Lot-et-Garonne', 'Pyrénées-Atlantiques', 'Deux-Sèvres', 'Vienne', 'Haute-Vienne'],
    'Occitanie': ['Ariège', 'Aude', 'Aveyron', 'Gard', 'Haute-Garonne', 'Gers', 'Hérault', 'Lot', 'Lozère', 'Hautes-Pyrénées', 'Pyrénées-Orientales', 'Tarn', 'Tarn-et-Garonne'],
    'Pays de la Loire': ['Loire-Atlantique', 'Maine-et-Loire', 'Mayenne', 'Sarthe', 'Vendée'],
    'Provence-Alpes-Côte d\'Azur': ['Alpes-de-Haute-Provence', 'Hautes-Alpes', 'Alpes-Maritimes', 'Bouches-du-Rhône', 'Var', 'Vaucluse']
}

Regions = [
    'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    # 'Corse',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]

Name2num = {
    'Ain': '01',
    'Aisne': '02',
    'Allier': '03',
    'Alpes-de-Haute-Provence': '04',
    'Hautes-Alpes': '05',
    'Alpes-Maritimes': '06',
    'Ardèche': '07',
    'Ardennes': '08',
    'Ariège': '09',
    'Aube': '10',
    'Aude': '11',
    'Aveyron': '12',
    'Bouches-du-Rhône': '13',
    'Calvados': '14',
    'Cantal': '15',
    'Charente': '16',
    'Charente-Maritime': '17',
    'Cher': '18',
    'Corrèze': '19',
    # 'Corse-du-Sud': '2A',
    # 'Haute-Corse': '2B',
    'Côte-d\'Or': '21',
    'Côtes-d\'Armor': '22',
    'Creuse': '23',
    'Dordogne': '24',
    'Doubs': '25',
    'Drôme': '26',
    'Eure': '27',
    'Eure-et-Loir': '28',
    'Finistère': '29',
    'Gard': '30',
    'Haute-Garonne': '31',
    'Gers': '32',
    'Gironde': '33',
    'Hérault': '34',
    'Ille-et-Vilaine': '35',
    'Indre': '36',
    'Indre-et-Loire': '37',
    'Isère': '38',
    'Jura': '39',
    'Landes': '40',
    'Loir-et-Cher': '41',
    'Loire': '42',
    'Haute-Loire': '43',
    'Loire-Atlantique': '44',
    'Loiret': '45',
    'Lot': '46',
    'Lot-et-Garonne': '47',
    'Lozère': '48',
    'Maine-et-Loire': '49',
    'Manche': '50',
    'Marne': '51',
    'Haute-Marne': '52',
    'Mayenne': '53',
    'Meurthe-et-Moselle': '54',
    'Meuse': '55',
    'Morbihan': '56',
    'Moselle': '57',
    'Nièvre': '58',
    'Nord': '59',
    'Oise': '60',
    'Orne': '61',
    'Pas-de-Calais': '62',
    'Puy-de-Dôme': '63',
    'Pyrénées-Atlantiques': '64',
    'Hautes-Pyrénées': '65',
    'Pyrénées-Orientales': '66',
    'Bas-Rhin': '67',
    'Haut-Rhin': '68',
    'Rhône': '69',
    'Haute-Saône': '70',
    'Saône-et-Loire': '71',
    'Sarthe': '72',
    'Savoie': '73',
    'Haute-Savoie': '74',
    'Paris': '75',
    'Seine-Maritime': '76',
    'Seine-et-Marne': '77',
    'Yvelines': '78',
    'Deux-Sèvres': '79',
    'Somme': '80',
    'Tarn': '81',
    'Tarn-et-Garonne': '82',
    'Var': '83',
    'Vaucluse': '84',
    'Vendée': '85',
    'Vienne': '86',
    'Haute-Vienne': '87',
    'Vosges': '88',
    'Yonne': '89',
    'Territoire de Belfort': '90',
    'Essonne': '91',
    'Hauts-de-Seine': '92',
    'Seine-Saint-Denis': '93',
    'Val-de-Marne': '94',
    'Val-d\'Oise': '95'
}


AREA = []
for area in CFG.area:
    AREA.extend(Administratives[area] )
# AREA

In [ ]:
data_path = './datas'
file_name = 'decoupage_administratif'


communes = gpd.read_feather(os.path.join(data_path,f'[{CFG.area_name}]{file_name}_osmid_centre.feather'))

communes.crs

In [ ]:
metro = Name2num.values()
communes = communes.query("dep in @metro")

Dirty fix on errors in osmid from previous notebook

# TODO : enquire why, move fix to prevoius notebook

In [ ]:
# MANUAL FIX FOR MISSING OSMID ON Nominatim resquests failures
communes.loc[5228 , ['osmid','x','y']] = (89493,45.34241,0.14665)
communes.loc[5229 , ['osmid','x','y']] = (63211,45.32810,-0.19941)
communes.loc[10916, ['osmid','x','y']] = (59355,44.000480,4.368443)
communes.loc[34954, ['osmid','x','y']] = (78755,48.21385,-0.21375)

In [ ]:
dup = communes[communes[['osmid']].duplicated()]
dup

In [ ]:
for n, idx in enumerate(dup.index):
    communes.loc[idx,'osmid'] = n + 10

In [ ]:
dup = communes[communes[['osmid']].duplicated()]
dup

In [ ]:
# # Test on a sample
# deps = ['56']  #+['51','52','08','10']
# communes = communes.query("dep in @deps")



In [ ]:


display(communes.head(1))
display(communes.info())
communes.shape

In [ ]:
communes[communes.x.isna()]

In [ ]:
communes['osmid'] = communes['osmid'].astype(int)
communes['osmid'] = communes['osmid'].astype(str)

In [ ]:
candidate_labels = joblib.load( os.path.join(CFG.data_path,f'[{CFG.area_name}]candidate_labels.pkl'))
labels2legend = joblib.load( os.path.join(CFG.data_path,f'[{CFG.area_name}]labels2legend.pkl'))

file_name = 'datatourisme-place-20230529-tagged'

tourism = gpd.read_feather(os.path.join(CFG.data_path,file_name + '.feather'))


In [ ]:
tourism.set_crs(crs=4326)

In [ ]:
tourism.crs

# Create Node geodataframe

In [ ]:
display(communes.head(1),tourism.head(1)) 

In [ ]:
tourism['label'] = tourism.Class.map(labels2legend) 

tourism = tourism[['label','geometry']]


In [ ]:
communes = communes[['osmid','x','y','codgeo','dep','reg','geometry']]

In [ ]:
# joblib.load( os.path.join(CFG.data_path, f'[{CFG.area_name}]Gnodes.pkl')).head(1)

In [ ]:
labels2legend.values()

In [ ]:
tourism.set_geometry('geometry')

In [ ]:
def get_tag_counts(geom):
    sub = tourism[tourism['geometry'].within(geom)]
    counts = sub.label.value_counts()
    return counts


counts = get_tag_counts(communes.iloc[15]['geometry'])
counts 

In [ ]:
for key , value in zip(counts.index, counts):
    display({key:value})

In [ ]:
communes = communes.sample(frac=1.0)

In [ ]:
from shapely import Point
from networkx import MultiDiGraph
G = MultiDiGraph(crs = 4326)

total = communes.shape[0]
for n, (i, row) in enumerate(communes.iterrows()):
    counts = get_tag_counts(row['geometry'])
    # display(counts)
    print(f'{n+1}/{total} ~ {(n + 1) / total:.2%} ', end=' '*50 + '\r')
    attr = {
        'y': float(row.x),
        'x': float(row.y),
        # 'geometry': Point(float(row.x),float(row.y)),
        'codgeo': row.codgeo, 
        'area': row['geometry']
    }
    for key , value in zip(counts.index, counts):
        attr.update({key:int(value)})

    G.add_node(
        row.osmid,
        **attr
    )
    # if n == 1000 :
        # break



In [ ]:
# Check
nodes = ox.graph_to_gdfs(G, nodes=True, edges=False)
display(nodes.info(),nodes.head(5),nodes.tail(5))
nodes.plot()
del nodes

In [ ]:
######## BACKUP POINT ################

# backup = G.copy()
# G = backup.copy()

In [ ]:
from sklearn.neighbors import BallTree

def get_nearest(src_points, candidates, k_neighbors=1):
    """Find nearest neighbors for all source points from a set of candidate points"""
    tree = BallTree(candidates, leaf_size=20, metric='haversine')
    distances, indices = tree.query(src_points, k=k_neighbors)
    if src_points.shape[0] > 1 :
        distances = distances.transpose()
        indices = indices.transpose()
    closest = indices[0]
    closest_dist = distances[0]
    return (closest, closest_dist)

In [ ]:
ANGLE = 0.04

print(f'Distance for geopandas buffer: {6371 * ANGLE:.2f}')

In [ ]:
from shapely import Point
import numpy as np

ANGLE = 0.04

error_edges = []
coord = ['x', 'y']
print(f"Distance for geopandas buffer: {6371 * ANGLE:.2f} (gros doute sur l\'unité).")
for n, (i, row) in enumerate(communes.iterrows()):
    try:
        commune_center = Point(row.y,row.x)

        sub_communes = communes[communes['geometry'].intersects(commune_center.buffer(ANGLE))]

    
        closests, distances = get_nearest(src_points= np.array(row[coord]).reshape(1, -1), 
                              candidates= sub_communes[coord],
                              k_neighbors= min(sub_communes.shape[0], 5))
        # print('>>>>>',row.codgeo, sub_communes.shape[0], closests)
        

        # Dirty fix for incomprehensible long edges
        # distances = [ x for x in distances if x <= ANGLE]
        # closest= [closest[i]for i in distances]
        # stays 2 edges with abnornal non cloness edges (in south-est)

        index = sub_communes.index
        for _id in closests:
            
            node_id = sub_communes.loc[index[_id],'osmid']
            # print(row.osmid,
            #     node_id)
            # start = row.osmid
            # end = node_id
            # line, length = get_path_attributes(Graph_routier, Nodes_routier,start,end)
            # # display(line, length)
            # # TODO: faire une fonction pour recuperér depuis le graph france le plus court chemin 
            # #       entre les deux nodes et concatener linestrings distances et durées    
            # attr = {
            #     'geometry': line,
            #     'length': length
            # }


            G.add_edge(
                row.osmid,
                node_id,
                0,
                # **attr
            )
    except:
        error_edges.append(n)       


    # if n > 1000 : break
    print(f'{n+1}/{total} ~ {(n + 1) / total:.2%} ', end=' '*50 + '\r')    


print(f'Errors edges: {error_edges}              ')    

In [ ]:
G.number_of_nodes(), G.number_of_edges()

In [ ]:
ox.plot.plot_graph(G, ax=None, figsize=(8, 8), bgcolor='#000000', node_color='r', node_size=.5, node_alpha=0.2, node_edgecolor='r', node_zorder=1, edge_color='blue', edge_linewidth=0.8, edge_alpha=0.2, show=True, close=False, save=False, filepath=None, dpi=300, bbox=None)


## Edges cleaning

    remove (u,u) edges
    remove edges with distance in Lambert projection > 100 km

In [ ]:
nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)

In [ ]:
# display(nodes.info(), nodes.head(5))

In [ ]:
# nodes['y'] = nodes.y.astype(float)
# nodes['x'] = nodes.x.astype(float)

In [ ]:
# points = []
# new_x =[]
# new_y =[]
# for x, y in zip(nodes.x, nodes.y):
#     points.append(Point(y,x))
#     new_x.append(y)
#     new_y.append(x)

In [ ]:
# nodes['geometry'] = points
# nodes.set_geometry('geometry')

In [ ]:
# nodes['x'] = new_x
# nodes['y'] = new_y

In [ ]:
# nodes.head(5)

In [ ]:
# projection to Lambert, calculate distance, filter on distance, reproject to WS84
display(edges.info(), edges.head(5))
edges = ox.projection.project_gdf(edges,  to_crs=2154, to_latlong=False)  
edges['dist'] = edges.geometry.length

display(edges.dist.min(), edges.dist.max() )

edges = edges.query("dist >0 and dist < 100000")

display(edges.reset_index().shape)

edges.drop('dist', axis=1, inplace=True)
edges = ox.projection.project_gdf(edges,  to_crs=4326, to_latlong=False)  


In [ ]:
G1 = ox.graph_from_gdfs(nodes, edges)

In [ ]:
ox.plot.plot_graph(G1, ax=None, figsize=(8, 8), bgcolor='#000000', node_color='r', node_size=.5, node_alpha=0.2, node_edgecolor='r', node_zorder=1, edge_color='blue', edge_linewidth=0.8, edge_alpha=0.2, show=True, close=False, save=False, filepath=None, dpi=300, bbox=None)

In [ ]:
joblib.dump(nodes,os.path.join(CFG.data_path, f'[{CFG.area_name}]Gnodes_communes.pkl'))
joblib.dump(edges,os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges_communes.pkl'))


In [ ]:
ox.folium.plot_graph_folium(G1, 
                            graph_map=None, 
                            popup_attribute=None, 
                            tiles='cartodbpositron', 
                            zoom=1, 
                            fit_bounds=True)

------------------------------------------------------

In [ ]:
# betweeness = nx.betweenness_centrality(
#     G1, 
#     # k=None, 
#     # normalized=True, 
#     # weight='distance', 
#     # endpoints=False, 
#     # seed=42
# )

# betweeness.describe()